In [ ]:
# ! pip install bastionlab polars

In [2]:
!wget -O diabetes_train.csv.gz https://github.com/scikit-learn/scikit-learn/raw/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz
!wget -O diabetes_target.csv.gz https://github.com/scikit-learn/scikit-learn/raw/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_target.csv.gz
!gzip -d diabetes_train.csv.gz
!gzip -d diabetes_target.csv.gz

--2023-01-04 15:03:55--  https://github.com/scikit-learn/scikit-learn/raw/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/scikit-learn/scikit-learn/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz [following]
--2023-01-04 15:03:55--  https://raw.githubusercontent.com/scikit-learn/scikit-learn/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7105 (6.9K) [application/octet-stream]
Saving

In [14]:
from bastionlab import Connection, Identity
import polars as pl

connection = Connection("localhost")

In [15]:
# load dataset
train_df = pl.read_csv(
    "diabetes_train.csv",
    sep=" ",
    has_header=False,
    new_columns=["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
)
target_df = pl.read_csv(
    "diabetes_target.csv",
    has_header=False,
    new_columns=["target"],
)

In [16]:
from bastionlab.polars.policy import Policy, Aggregation, Log

policy = Policy(
    safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log(), savable=False
)
train_rdf = connection.client.polars.send_df(train_df)
target_rdf = connection.client.polars.send_df(target_df)

In [19]:
# Perform linear regression on `diabetes` dataset.
from bastionlab.linfa.trainers import LinearRegression
from bastionlab.polars import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(
    train_rdf, target_rdf, test_size=0.2, shuffle=True
)


train_X = train_X.to_array()
train_Y = train_Y.to_array()

lr = LinearRegression()

lr.fit(train_X, train_Y)

y_pred = lr.predict(test_X)

result = pl.DataFrame(
    {"Actual": test_Y.collect().fetch().to_series(), "Predict": y_pred.to_series()}
)
result

shape: (10, 1)
┌──────────┐
│ column_0 │
│ ---      │
│ f64      │
╞══════════╡
│ 60.0     │
├╌╌╌╌╌╌╌╌╌╌┤
│ 2.0      │
├╌╌╌╌╌╌╌╌╌╌┤
│ 28.2     │
├╌╌╌╌╌╌╌╌╌╌┤
│ 112.0    │
├╌╌╌╌╌╌╌╌╌╌┤
│ ...      │
├╌╌╌╌╌╌╌╌╌╌┤
│ 42.0     │
├╌╌╌╌╌╌╌╌╌╌┤
│ 4.0      │
├╌╌╌╌╌╌╌╌╌╌┤
│ 4.9836   │
├╌╌╌╌╌╌╌╌╌╌┤
│ 93.0     │
└──────────┘
Reason: Cannot fetch a result DataFrame that does not aggregate at least 10 rows of DataFrame 16bd3d29-6c16-4930-828e-794790492179.

A notification has been sent to the data owner. The request will be pending until the data owner accepts or denies it or until timeout seconds elapse.


KeyboardInterrupt: 